In [1]:
    if __name__ == '__main__':
        %reload_ext pidgin
        %reload_ext pidgin.application

In [2]:
    import functools, traitlets, dataclasses, IPython
    from pidgin import formatter, weave, shell
    try: 
        import ipywidgets
        widget_from_abbrev = ipywidgets.interaction.interactive.widget_from_abbrev
        default = dict((
            (tuple, widget_from_abbrev), 
            (str, widget_from_abbrev),
            (int, widget_from_abbrev),
            (float, widget_from_abbrev),
            (list, widget_from_abbrev),
            (dict, widget_from_abbrev),
            (set, widget_from_abbrev),
        ))
        container = ipywidgets.VBox
    except:
        default = {}
        class VBox(traitlets.HasTraits):
            children = traitlets.List()
        container = VBox

In [3]:
    class Manager(shell.Shell, IPython.core.formatters.BaseFormatter):
>>> app = IPython.get_ipython()
>>> app.manager
<...Manager object at 0x...>
>>> with IPython.utils.capture.capture_output() as out:
...     IPython.display.display(app)
>>> out.outputs[0].data
{...application/vnd.jupyter.widget-view+json...}

        container = container()
        annotations = traitlets.Dict()
        widgets = traitlets.Dict()
        parent = traitlets.Instance(IPython.InteractiveShell)
        type_printers = traitlets.Dict(default)
        
        def run_cell(Manager, *args, **kwargs):
            app = Manager.parent
            __annotations__ = app.user_ns.get('__annotations__', {})
            for name, value in __annotations__.items():
                if type(value) != type(Manager.annotations.get(name, None)):
                    Manager.annotations[name] = value
                    
                    Manager.widgets[name] = Manager.lookup(value)(value)
                    Manager.widgets[name].description = Manager.widgets[name].description or name
                    
                    object = Manager.user_ns.get(name, Manager.widgets[name].value)
                    Manager.parent.add_traits(**{name: traitlets.Any(default_value=object)})
                    traitlets.link((Manager.parent, name), (Manager.widgets[name], 'value'))
                    Manager.parent.observe(functools.partial(Manager.observer, name), names=name)
                Manager.observer(name)
                if hasattr(Manager.widgets[name], 'options'):
                    Manager.widgets[name].options = type(Manager.widgets[name].options)(value)
                    
                    
        def observer(Manager, key, change=None):
>>> app = IPython.get_ipython()
>>> app.user_ns['__annotations__'] = app.user_ns.get('__annotations__', {})
>>> app.user_ns.update(__annotations__="""---
... {x: 10, y: 20, m: 2}""")
>>> app.run_cell('')
<ExecutionResult...>
>>> traitlets.dlink((app, 'x'), (app, 'y'), lambda x: x**app.m)
<traitlets.traitlets.directional_link...>
>>> app.x = 3
>>> assert app.y == app.x**app.m, "The link didn't fire."

            Manager.user_ns[key] = Manager.widgets[key].value = change['new'] if change else Manager.user_ns.get(key, Manager.widgets[key].value)
            setattr(Manager.parent, key, Manager.user_ns[key])
            
        def _ipython_display_(Manager): 
            Manager.container.children = list(Manager.widgets.values())
            IPython.display.display(Manager.container)

In [4]:
    def load_ipython_extension(app):
>>> %reload_ext pidgin.application
        
        app.manager = Manager(user_module=app.user_module, parent=app, parents=[app])
        app.user_ns['app'] = app
        app._ipython_display_ = app.manager._ipython_display_
        app.events.register('post_run_cell', app.manager)

In [5]:
    def unload_ipython_extension(app):
>>> %unload_ext pidgin.application
>>> with IPython.utils.capture.capture_output() as out:
...     IPython.display.display(app)
>>> out.outputs[0].data
{...ZMQInteractiveShell...}
>>> %load_ext pidgin.application
        
        del app._ipython_display_
        pops = []
        for i, object in enumerate(app.events.callbacks['post_run_cell']):
            if isinstance(object, Manager): pops.append(i)
        for i in reversed(pops): app.events.callbacks['post_run_cell'].pop(i)
    if __name__ == '__main__': load_ipython_extension(get_ipython())